In [1]:
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

submission = pd.read_csv('data/sample_submission.csv')

In [2]:
# 원핫 인코딩
train_one = pd.get_dummies(train)
test_one = pd.get_dummies(test)
train_data, train_target = train_one.drop(['index', 'INVC_CONT'],axis=1), train_one['INVC_CONT']
test_data = test_one.drop(['index'],axis=1)

In [3]:
train_data['SEND_CODE'] = train_data['SEND_SPG_INNB'].apply(lambda x: str(x)[:5])
train_data['REC_CODE'] = train_data['REC_SPG_INNB'].apply(lambda x: str(x)[:5])

test_data['SEND_CODE'] = test_data['SEND_SPG_INNB'].apply(lambda x: str(x)[:5])
test_data['REC_CODE'] = test_data['REC_SPG_INNB'].apply(lambda x: str(x)[:5])

train_data.drop(['SEND_SPG_INNB','REC_SPG_INNB'], axis=1, inplace=True)
test_data.drop(['SEND_SPG_INNB','REC_SPG_INNB'], axis=1, inplace=True)

In [4]:
from tpot import TPOTRegressor

tpot_params = dict({
    'generations':10,
    'population_size':100,
    'subsample':1,
    'verbosity':2,
    'random_state':42,
    'n_jobs':-1,
    'early_stop':100,
    'use_dask':True,
    'warm_start':True,
    'memory':'auto',
    'periodic_checkpoint_folder':'./tpot_checkpoint/'
})

In [5]:
tpot = TPOTRegressor(**tpot_params)
for i in range(100):
    tpot.fit(train_data, train_target)
    tpot.export(f'tpot_best_pareto/tpot_pipeline_new_{i}.py')
    results = tpot.predict(test_one)
    submission['INVC_CONT'] = results
    submission.to_csv(f'submission/autosk_v{i}.csv', index=False)


Generation 1 - Current best internal CV score: -29.194453074268644

Generation 2 - Current best internal CV score: -28.933637223071656

Generation 3 - Current best internal CV score: -28.933538551446627
Optimization Progress:  36%|███▋      | 401/1100 [1:21:00<5:03:28, 26.05s/pipeline]